In [1]:
import mne
import numpy as np
import pyxdf
from datetime import datetime
from collections import Counter

In [2]:
filepath = 'C:\PilotData\sub-Pilooto1_ses-S001_task-Default_run-001_eeg.xdf'
data, header = pyxdf.load_xdf(filepath)

In [46]:
class XDFData:
    def __init__(self, filepath=None) -> None:
        self.rawData = None
        self.filepath = filepath
        self.data = data
        self.header = header
        self.setupData()
        self.printInfo()
        self.createMNEObjectForEEG()
        self.makeAnnotations()
    def loadXdfData(self):
        self.data, self.header = pyxdf.load_xdf(filepath)
    def setupData(self,):
        self.eegChannelNames = []
        self.measDate = self.header['info']['datetime'][0]
        self.markers = self.data[1]['time_series']
        self.markers.pop()
        self.markers = [marker[0] for marker in self.markers]
        self.markersTimestamps = self.data[1]['time_stamps']
        self.eegData = self.data[2]['time_series']
        self.eegSamplingFrequency = int(float(self.data[2]['info']['nominal_srate'][0]))
        self.eegTimestamps = self.data[2]['time_stamps']
        self.audioData = self.data[3]['time_series']
        self.audioTimestamps = self.data[3]['time_stamps']
        self.audioSamplingFrequency = int(float(self.data[3]['info']['nominal_srate'][0]))
        channelNames = data[2]['info']['desc'][0]['channels'][0]['channel']
        for item in channelNames:
            self.eegChannelNames.append(item['label'][0]) 
    def printInfo(self):
        print(f'No of Markers: {len(self.markers)} No .of Marker Timestamps: {self.markersTimestamps.shape[0]}')
        print(f'EEG data Shape: {self.eegData.shape} No .of eeg Timestamps: {self.eegTimestamps.shape[0]}')
        print(f'Audio data Shape: {self.audioData.shape} No .of audio Timestamps: {self.audioTimestamps.shape[0]}')
        print('EEG Channels:', self.eegChannelNames)
        print(f'Sampling Frequency ::: EEG: {self.eegSamplingFrequency}, Audio: {self.audioSamplingFrequency}')
    def createMNEObjectForEEG(self):
        meas_date = datetime.strptime(self.measDate, '%Y-%m-%dT%H:%M:%S%z')
        meas_date = (int(meas_date.timestamp()), int(meas_date.microsecond))
        info = mne.create_info(
            ch_names=self.eegChannelNames, 
            sfreq=self.eegSamplingFrequency, 
            ch_types='eeg',
        )
        info.set_meas_date(meas_date)
        self.rawEegMNEData = mne.io.RawArray(self.eegData.T, info)
    def getOnsetCodesForAnnotations(self):
        codes = []
        onset = []
        description = []
        duration = []
        markers = self.markers
        markersTimestamps = self.markersTimestamps - self.eegTimestamps[0]
        for index in range(len(markers)-1):
            marker = markers[index]
            code = ''
            if 'Silent' in marker:
                code += '10' 
            if 'Real' in marker:
                code += '11' 
            if 'Word' in marker:
                code += '12' 
            if 'Syllable' in marker:
                code += '13' 
            if 'Practice' in marker:
                code += '14' 
            if 'Experiment' in marker:
                code += '15' 
            if 'Start' in marker:
                code += '16' 
            if 'End' in marker:
                code += '17' 
            if 'Fixation' in marker:
                code += '18' 
            elif 'Stimulus' in marker:
                code += '19' 
            elif 'ISI' in marker:
                code += '20' 
            elif 'ITI' in marker:
                code += '21' 
            elif 'Speech' in marker:
                code += '22' 
            else:
                print(marker)
            
            if 'Audio' in marker:
                code += '23'
            elif 'Text' in marker:
                code += '24'
            elif 'Speech' in marker:
                code += '25'

            codes.append(code)
            onset.append(markersTimestamps[index])
            description.append(marker)
            duration.append(markersTimestamps[index+1]-markersTimestamps[index])

        return onset, codes, duration

    def makeAnnotations(self):
        onset, codes, duration = self.getOnsetCodesForAnnotations()
        rawMNEWithAnnotations = self.rawEegMNEData.copy()
        rawMNEWithAnnotations.set_annotations(mne.Annotations(onset=onset, description=codes, duration=duration))

        self.rawMNEWithAnnotations = rawMNEWithAnnotations

In [48]:
obj = XDFData()


No of Markers: 5314 No .of Marker Timestamps: 5322
EEG data Shape: (3814127, 64) No .of eeg Timestamps: 3814127
Audio data Shape: (183075840, 1) No .of audio Timestamps: 183075840
EEG Channels: ['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'EOG1', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'EOG2', 'CP6', 'CP2', 'Cz', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'C1', 'C5', 'TP7', 'CP3', 'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2', 'CPz', 'CP4', 'TP8', 'C6', 'C2', 'FC4', 'FT8', 'F6', 'AF8', 'AF4', 'F2', 'FCz']
Sampling Frequency ::: EEG: 1000, Audio: 48000
Creating RawArray with float64 data, n_channels=64, n_times=3814127
    Range : 0 ... 3814126 =      0.000 ...  3814.126 secs
Ready.


<Annotations | 5313 segments: 1012141618 (2), 101214161823 (4), ...>

In [ ]:
def createTwoEvents(XDFObject):
    events, description =  mne.events_from_annotations(XDFObject.rawMNEWithAnnotations)

In [53]:
events, description = mne.events_from_annotations(obj.rawMNEWithAnnotations)

Used Annotations descriptions: ['1012141618', '101214161823', '101214161824', '1012141619', '101214161923', '101214161924', '1012141620', '101214162023', '101214162024', '1012141621', '101214162123', '101214162124', '101214162223', '101214162224', '101214162225', '1012141718', '101214171823', '101214171824', '1012141719', '101214171923', '101214171924', '1012141720', '101214172023', '101214172024', '1012141721', '101214172123', '101214172124', '101214172223', '101214172224', '101214172225', '1012151618', '101215161823', '101215161824', '1012151619', '101215161923', '101215161924', '1012151620', '101215162023', '101215162024', '1012151621', '101215162123', '101215162124', '101215162223', '101215162224', '101215162225', '1012151718', '101215171823', '101215171824', '1012151719', '101215171923', '101215171924', '1012151720', '101215172023', '101215172024', '1012151721', '101215172123', '101215172124', '101215172223', '101215172224', '101215172225', '101314161823', '101314161824', '1013141

In [58]:
description

{'1012141618': 1,
 '101214161823': 2,
 '101214161824': 3,
 '1012141619': 4,
 '101214161923': 5,
 '101214161924': 6,
 '1012141620': 7,
 '101214162023': 8,
 '101214162024': 9,
 '1012141621': 10,
 '101214162123': 11,
 '101214162124': 12,
 '101214162223': 13,
 '101214162224': 14,
 '101214162225': 15,
 '1012141718': 16,
 '101214171823': 17,
 '101214171824': 18,
 '1012141719': 19,
 '101214171923': 20,
 '101214171924': 21,
 '1012141720': 22,
 '101214172023': 23,
 '101214172024': 24,
 '1012141721': 25,
 '101214172123': 26,
 '101214172124': 27,
 '101214172223': 28,
 '101214172224': 29,
 '101214172225': 30,
 '1012151618': 31,
 '101215161823': 32,
 '101215161824': 33,
 '1012151619': 34,
 '101215161923': 35,
 '101215161924': 36,
 '1012151620': 37,
 '101215162023': 38,
 '101215162024': 39,
 '1012151621': 40,
 '101215162123': 41,
 '101215162124': 42,
 '101215162223': 43,
 '101215162224': 44,
 '101215162225': 45,
 '1012151718': 46,
 '101215171823': 47,
 '101215171824': 48,
 '1012151719': 49,
 '101215

In [57]:
newEvents = []
for index in range(len(events)):
    if '23' in description[index]:
        pass
for event, desc in zip(events, description):
    print(event, desc)

KeyError: 0